In [2]:
cd ..

/Users/svenvanderburg/projects/modys-video


# Experiment 1.1 Explore models and approaches for deep learning
Finetune models on left leg amplitude prediction with lying videos as input.
For now this is just to quickly test the settings that Shankara got out of previous experiments, i.e.:

* Cutoff first 50 frames
* Interpolate when likelihood is below 0.7
* Use opposite bodypart if all likelihood is below 0.7 for a bodypart
* Use a standard scaler to scale
    

We don't really know whether this is the best set of hyperparameters. We can further experiment with the following hyperparameters:
* Use a StandardScaler
* Include likelihood
* Use different model architectures
* Use interpolation
* write one function that takes in parameters

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as keras


from src.helpers import read_scores
from src.data_generators import RawDataGenerator
from src.data_selection import MultipleScoreSelector
from src.ai_func import cross_validation_generator
from src.settings import LYING_VIDEOS_DATA_FOLDER, SITTING_VIDEOS_DATA_FOLDER, DATA_FOLDER


from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Lambda
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.advanced_activations import LeakyReLU

%load_ext autoreload
%autoreload 2

In [4]:
scores_df = read_scores(DATA_FOLDER / 'data_Scoring_DIS_proximal_trunk_V1.1.xlsx')

## Definitions

In [5]:
SCORES_TO_USE = ['D_LLP_R_tA_pscore']
SCORER_TO_USE = 1

## Pipeline for training a deep neural network

### Define model architecture (here: simple CNN)

In [6]:
# Generate some dev data to get X shape
selector = MultipleScoreSelector(scores_to_use=SCORES_TO_USE, scorer_to_use=SCORER_TO_USE)
dev_selection = selector.transform(scores_df)
dev_generator = RawDataGenerator(dev_selection, videos_folder=LYING_VIDEOS_DATA_FOLDER, cutoff=50, likelihood=0.7)
X, y = dev_generator.__getitem__(0)
n_timesteps, n_features = (X.shape[1], X.shape[2])

/Users/svenvanderburg/projects/modys-video/src/data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
/Users/svenvanderburg/projects/modys-video/src/data_selection.py:34: UserWarning: Dropping rows ['090' '058' '048'] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


In [7]:
n_outputs = len(SCORES_TO_USE)

def get_model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(1))
    return model


def get_model_old():
    # simple CNN
    input_layer = keras.layers.Input(shape=(n_timesteps,n_features))
    norm_layer = keras.layers.BatchNormalization()(input_layer)
    cnn_layer = keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(norm_layer)
    cnn_layer = keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.MaxPooling1D(pool_size=2)(cnn_layer)
    cnn_layer = keras.layers.Dropout(0.7)(cnn_layer)
    cnn_layer = keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.MaxPooling1D(pool_size=2)(cnn_layer)
    cnn_layer = keras.layers.Dropout(0.7)(cnn_layer)
    cnn_layer = keras.layers.Flatten()(cnn_layer)
    cnn_layer = keras.layers.Dense(100)(cnn_layer)
    output_layer = keras.layers.Dense(n_outputs)(cnn_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)
get_model().summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 449, 64)           3520      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 447, 64)           12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 223, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 219, 32)           10272     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 215, 32)           5152      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 107, 32)           0         
_________________________________________________________________
flatten (Flatten)            (None, 3424)              0

2021-11-29 14:36:47.061625: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Train network

In [13]:
data_generation_params = {
    'videos_folder': LYING_VIDEOS_DATA_FOLDER,
    'cutoff': 50,
    'likelihood': 0.7
}    

def train_model(train_generator):
    model = get_model()
    model.compile(loss='mae', optimizer=keras.optimizers.Adam())
    model.fit(train_generator, epochs=30)
    return model

def train_cross_val(cross_val):
    y_pred = []
    y_test = []
    for i_split, (train_scores, test_scores) in enumerate(cross_val):
        print(f'Fitting for 5-fold split {i_split}')
        train_generator = RawDataGenerator(train_scores, **data_generation_params)
        scaler = train_generator._get_scaler()
        test_generator = RawDataGenerator(test_scores, **data_generation_params)
        model = train_model(train_generator)
        y_pred.append(model.predict(test_generator))
        y_test.append(test_scores)
        break # This results in training only for one of the 5 folds
    y_pred = np.vstack(y_pred)
    y_test = pd.concat(y_test)
    return y_test, y_pred

def evaluate(y_test, y_pred):
    results = []
    for i_score, column in enumerate(y_test):
        mae = mean_absolute_error(y_test.iloc[:, i_score], y_pred[:, i_score])
        results.append({'score': column, 'mae': mae})
    return pd.DataFrame(results)

In [14]:
print(f'Training model for scorer {SCORER_TO_USE}')
selector = MultipleScoreSelector(scores_to_use=SCORES_TO_USE, scorer_to_use=SCORER_TO_USE)
selected_data = selector.transform(scores_df)
cross_val = cross_validation_generator(selected_data)
y_test, y_pred = train_cross_val(cross_val)

Training model for scorer 1


/Users/svenvanderburg/projects/modys-video/src/data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
/Users/svenvanderburg/projects/modys-video/src/data_selection.py:34: UserWarning: Dropping rows ['090' '058' '048'] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/30
49/49 [==============================] - 3s 58ms/step - loss: 59.8023
Epoch 2/30
49/49 [==============================] - 3s 60ms/step - loss: 3.4945
Epoch 3/30
49/49 [==============================] - 3s 58ms/step - loss: 0.5678
Epoch 4/30
49/49 [==============================] - 3s 58ms/step - loss: 0.2379
Epoch 5/30
49/49 [==============================] - 3s 60ms/step - loss: 0.2420
Epoch 6/30
49/49 [==============================] - 3s 58ms/step - loss: 0.2554
Epoch 7/30
49/49 [==============================] - 3s 58ms/step - loss: 0.2411
Epoch 8/30
49/49 [==============================] - 3s 59ms/step - loss: 0.2511
Epoch 9/30
49/49 [==============================] - 3s 58ms/step - loss: 0.2385
Epoch 10/30
49/49 [==============================] - 3s 59ms/step - loss: 0.2378
Epoch 11/30
49/49 [==============================] - 3s 57ms/step - loss: 0.2335
Epoch 12/30
49/49 [==============================] - 3s 58ms/step - loss: 0.2568
Epoch 13/

In [15]:
print(evaluate(y_test, y_pred))

               score      mae
0  D_LLP_R_tA_pscore  0.10663
